In [1]:
import numpy as np
import pandas as pd


In [2]:
credits= pd.read_csv('tmdb_5000_credits.csv')
movies=pd.read_csv('tmdb_5000_movies.csv')

In [10]:
credits.head()
credits=credits.rename(columns={'movie_id':'id'})

In [17]:
movies_df_merge=movies.merge(credits,on='id')
movies_df_merge=movies_df_merge.drop(columns=['homepage','title_x','title_y','status','production_companies'])

In [19]:
movies_df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   id                    4803 non-null   int64  
 3   keywords              4803 non-null   object 
 4   original_language     4803 non-null   object 
 5   original_title        4803 non-null   object 
 6   overview              4800 non-null   object 
 7   popularity            4803 non-null   float64
 8   production_countries  4803 non-null   object 
 9   release_date          4802 non-null   object 
 10  revenue               4803 non-null   int64  
 11  runtime               4801 non-null   float64
 12  spoken_languages      4803 non-null   object 
 13  tagline               3959 non-null   object 
 14  vote_average          4803 non-null   float64
 15  vote_count           

In [21]:
movies_df_merge.head()['overview']

0    In the 22nd century, a paraplegic Marine is di...
Name: overview, dtype: object

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv= TfidfVectorizer(min_df=3, max_features= None, strip_accents='unicode',analyzer='word', token_pattern='r\w{1,}',
                     ngram_range=(1,3), stop_words='english')

movies_df_merge['overview']=movies_df_merge['overview'].fillna('')

In [27]:
tfv_matrix= tfv.fit_transform(movies_df_merge['overview'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ral', 'rd', 'rds', 'ready', 'reafter', 'reas', 'reby', 'red', 'ree', 'refore', 'rehand', 'rein', 'reover', 'rest', 'reupon', 'rever', 'rhaps', 'ribe', 'ring', 'rious', 'rly', 'rmer', 'rmerly', 'rom', 'ront', 'ross', 'rough', 'roughout', 'round', 'rs', 'rself', 'rselves', 'rst', 'rt', 'rtheless', 'rther', 'rty', 'ru', 'rwards', 'rwise', 'ry', 'ryone', 'rything', 'rywhere'] not in stop_words.
  warnings.warn(


In [30]:
tfv_matrix.shape

(4803, 3626)

In [31]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig= sigmoid_kernel(tfv_matrix,tfv_matrix)

In [32]:
sig[0]

array([0.76170995, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159906,
       0.76159577])

In [33]:
indices=pd.Series(movies_df_merge.index,index=movies_df_merge['original_title']).drop_duplicates()

In [34]:
indices

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [35]:
def give_rec(title, sig=sig):
  idx=indices[title]
  sig_scores=list(enumerate(sig[idx]))
  sig_scores =sorted(sig_scores, key= lambda x: x[1], reverse=True)
  sig_scores=sig_scores[1:11]
  movie_indices=[i[0] for i in sig_scores]
  return movies_df_merge['original_title'].iloc[movie_indices]

In [36]:
give_rec('Avatar')

232                    The Wolverine
4263                   The Dog Lover
3724                   Falcon Rising
3568    20,000 Leagues Under the Sea
2130                    The American
1013                        Child 44
4413                       Childless
3374                      Veer-Zaara
4630                      Compliance
4670                         Mad Max
Name: original_title, dtype: object